# 🚀 OPTIMIZED DPO TRAINING - Maximum GPU Utilization

**Optimizations:**
- Batch size 4 (was 1) → 4x more GPU work
- Gradient accumulation 4 (was 16) → Less waiting
- Multi-GPU with accelerate
- Parallel data loading
- Auto-save with ZIP backup
- Checkpoints every epoch

**Expected Runtime:** 2-3 hours (was 10 hours)
**GPU Utilization:** 70-85% (was 35%)

---

In [ ]:
# Cell 1: Environment Setup
import os
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['TRL_USE_RICH'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

!pip install -q -U trl peft bitsandbytes accelerate transformers datasets

import warnings
warnings.filterwarnings('ignore')

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.1f} GB")

print("\n✅ Environment ready")

In [ ]:
# Cell 2: Load Dataset & Model (Optimized)
import json
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

print("="*80)
print("LOADING DATASET & MODEL")
print("="*80)

# Find dataset
DATA_FILE = None
for p in ["/kaggle/input/final-dpo-dataset/final_dpo_dataset.json",
          "/kaggle/input/dpo-dataset/final_dpo_dataset.json",
          "/kaggle/input/finaldpodataset/final_dpo_dataset.json"]:
    if os.path.exists(p): DATA_FILE = p; break

if not DATA_FILE:
    # List available inputs
    print("Available inputs:")
    for item in os.listdir("/kaggle/input/"):
        print(f"  {item}")
    raise FileNotFoundError("Upload final_dpo_dataset.json!")

print(f"\n📂 Dataset: {DATA_FILE}")

# Load data
with open(DATA_FILE) as f:
    data = json.load(f)

print(f"   Total pairs: {len(data)}")

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(data)
print(f"✅ Dataset loaded: {len(dataset)} pairs")

# Load model with optimizations
print(f"\n📥 Loading SmolLM2-360M-Instruct (optimized)...")

MODEL_NAME = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Load in bfloat16 for faster training
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager"  # Compatible with all GPUs
)

print(f"✅ Model loaded")
print(f"   Parameters: {model.num_parameters() / 1e6:.1f}M")
print(f"   Device: {model.device}")

In [ ]:
# Cell 3: Configure LoRA (Optimized)
print("\n" + "="*80)
print("LORA CONFIGURATION")
print("="*80)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # More layers = better
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"\n📊 LoRA Stats:")
print(f"   Trainable: {trainable / 1e6:.2f}M ({100*trainable/total:.2f}%)")
print(f"   Total: {total / 1e6:.1f}M")
print(f"\n✅ LoRA configured")

In [ ]:
# Cell 4: DPO Training Configuration (OPTIMIZED FOR SPEED)
from trl import DPOConfig, DPOTrainer

print("\n" + "="*80)
print("DPO TRAINING CONFIGURATION (OPTIMIZED)")
print("="*80)

# Calculate optimal batch settings
# Goal: Maximize GPU utilization while maintaining quality
# T4 has 16GB VRAM, we can use batch_size=4 with 360M model

BATCH_SIZE = 4           # Was 1 → 4x more GPU work per step
GRAD_ACCUM = 4           # Was 16 → Less waiting between updates
EFFECTIVE_BATCH = BATCH_SIZE * GRAD_ACCUM  # = 16 (same as before)

training_args = DPOConfig(
    # Core DPO
    beta=0.1,
    
    # OPTIMIZED batch settings
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    
    # Training params
    num_train_epochs=3,            # 3 epochs is enough with good data
    learning_rate=5e-6,            # Slightly higher for faster convergence
    
    # Length
    max_length=512,
    max_prompt_length=256,
    
    # Optimization
    optim="adamw_torch_fused",     # Faster optimizer
    warmup_ratio=0.1,
    
    # Mixed precision
    bf16=True,
    
    # Parallel data loading
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    
    # Logging & Checkpointing
    logging_steps=25,
    save_strategy="epoch",
    save_total_limit=2,
    output_dir="/kaggle/working/dpo_checkpoints",
    
    # Disable wandb
    report_to="none",
    
    # Gradient checkpointing for memory efficiency
    gradient_checkpointing=True
)

# Calculate expected training time
total_steps = (len(dataset) * training_args.num_train_epochs) // EFFECTIVE_BATCH
estimated_hours = total_steps * 4 / 3600  # ~4 seconds per step with optimization

print(f"\n📋 Configuration:")
print(f"   Batch size: {BATCH_SIZE} (was 1)")
print(f"   Gradient accumulation: {GRAD_ACCUM} (was 16)")
print(f"   Effective batch: {EFFECTIVE_BATCH}")
print(f"   Total steps: ~{total_steps}")
print(f"   Estimated time: ~{estimated_hours:.1f} hours")
print(f"   Checkpoints: Every epoch")
print(f"\n✅ Configuration ready")

In [ ]:
# Cell 5: Initialize & Train
import time

print("\n" + "="*80)
print("INITIALIZING DPO TRAINER")
print("="*80)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer
)

print(f"✅ Trainer initialized")
print(f"\n" + "="*80)
print("🚀 STARTING OPTIMIZED DPO TRAINING")
print("="*80)
print(f"\n⏱️  Estimated: 2-3 hours")
print(f"📊 Dataset: {len(dataset)} pairs")
print(f"🎯 GPU utilization target: 70-85%")
print(f"💾 Checkpoints: Every epoch (auto-saved)\n")

start_time = time.time()

# Train
trainer.train()

training_time = (time.time() - start_time) / 3600

print(f"\n" + "="*80)
print(f"✅ TRAINING COMPLETE in {training_time:.2f} hours")
print("="*80)

In [ ]:
# Cell 6: SAVE MODELS (Critical - Auto-save with verification)
import shutil
import os

print("\n" + "="*80)
print("💾 SAVING MODELS (Auto-save with verification)")
print("="*80)

LORA_PATH = "/kaggle/working/dpo_lora_adapter"
MERGED_PATH = "/kaggle/working/dpo_merged_model"
ZIP_PATH = "/kaggle/working/aligned_model.zip"

# Step 1: Save LoRA adapter
print(f"\n1️⃣ Saving LoRA adapter...")
model.save_pretrained(LORA_PATH)
tokenizer.save_pretrained(LORA_PATH)

if os.path.exists(LORA_PATH):
    files = os.listdir(LORA_PATH)
    print(f"   ✅ LoRA saved: {len(files)} files")
else:
    print(f"   ❌ LoRA save FAILED!")

# Step 2: Merge LoRA with base model
print(f"\n2️⃣ Merging LoRA with base model...")
merged_model = model.merge_and_unload()
merged_model.save_pretrained(MERGED_PATH)
tokenizer.save_pretrained(MERGED_PATH)

if os.path.exists(MERGED_PATH):
    files = os.listdir(MERGED_PATH)
    total_size = sum(os.path.getsize(os.path.join(MERGED_PATH, f)) for f in files) / (1024**2)
    print(f"   ✅ Merged model saved: {len(files)} files, {total_size:.1f} MB")
else:
    print(f"   ❌ Merged model save FAILED!")

# Step 3: Create ZIP backup
print(f"\n3️⃣ Creating ZIP backup...")
shutil.make_archive("/kaggle/working/aligned_model", 'zip', MERGED_PATH)

if os.path.exists(ZIP_PATH):
    zip_size = os.path.getsize(ZIP_PATH) / (1024**2)
    print(f"   ✅ ZIP created: {zip_size:.1f} MB")
else:
    print(f"   ❌ ZIP creation FAILED!")

# Step 4: Verify all outputs
print(f"\n4️⃣ Verifying all outputs...")
print(f"\nFiles in /kaggle/working/:")
for item in os.listdir("/kaggle/working/"):
    full_path = os.path.join("/kaggle/working/", item)
    if os.path.isdir(full_path):
        subfiles = len(os.listdir(full_path))
        print(f"   📁 {item}/ ({subfiles} files)")
    else:
        size = os.path.getsize(full_path) / (1024**2)
        print(f"   📄 {item} ({size:.1f} MB)")

print(f"\n" + "="*80)
print("✅ ALL MODELS SAVED SUCCESSFULLY")
print("="*80)

In [ ]:
# Cell 7: Quick Evaluation
import random
from tqdm.auto import tqdm

print("\n" + "="*80)
print("📊 QUICK EVALUATION")
print("="*80)

# Sample 100 pairs for quick eval
eval_sample = random.sample(data, min(100, len(data)))

def score_response(prompt, response):
    text = f"{prompt}\n\nResponse: {response}"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(merged_model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = merged_model(**inputs, labels=inputs["input_ids"])
        return -outputs.loss.item()

correct = 0
for item in tqdm(eval_sample, desc="Evaluating"):
    chosen_score = score_response(item['prompt'], item['chosen'])
    rejected_score = score_response(item['prompt'], item['rejected'])
    if chosen_score > rejected_score:
        correct += 1

accuracy = 100 * correct / len(eval_sample)

print(f"\n✅ Preference Accuracy: {accuracy:.1f}%")
print(f"   (Baseline was 96.8% with 411 pairs)")

if accuracy > 95:
    print(f"   🎉 EXCELLENT - Training successful!")
elif accuracy > 85:
    print(f"   ✅ Good performance")
else:
    print(f"   ⚠️ Lower than expected")

In [ ]:
# Cell 8: Final Summary & Download Instructions
print("\n" + "="*80)
print("🎉 TRAINING COMPLETE - DOWNLOAD YOUR MODEL")
print("="*80)

print(f"\n📊 Results:")
print(f"   Training time: {training_time:.2f} hours")
print(f"   Preference accuracy: {accuracy:.1f}%")
print(f"   Dataset: {len(data)} pairs")

print(f"\n📥 Files to Download:")
print(f"   1. aligned_model.zip ({zip_size:.1f} MB) ← Main model")
print(f"   2. dpo_merged_model/ ← Full folder")
print(f"   3. dpo_lora_adapter/ ← LoRA only")

print(f"\n🔧 How to Download:")
print(f"   Option 1: Right sidebar → Output → Click files")
print(f"   Option 2: After 'Save Version' → Notebook page → Output tab")

print(f"\n⚠️ IMPORTANT: Click 'Save Version' NOW to persist outputs!")
print(f"   (Top right button → Save & Run All)")

print(f"\n" + "="*80)
print("✨ Production-grade Gricean-aligned model ready!")
print("="*80)